##### Main task
Ми хочемо зрозуміти, як користувачі з різних країн поводяться з часом:
– хто краще утримується \
– хто приносить більше цінності \
– де є проблеми продукту або локалізації 

Для цього треба:
> 1. Підготувати данні: очистка + створити InvoiceMonth
> 2. Визначити місяць входу користувача 
> 3. Обираємо топ країни
> 4. Рахуємо користувачів 
> 5. Рахуємо retention 
> 6. Будуємо heatmap

In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt


In [24]:
df = pd.read_excel('data/Online Retail.xlsx') # !! треба змінити на свій шлях !!

In [25]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [27]:
# Data Cleaning 
df.drop_duplicates(inplace=True)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0) & (df['CustomerID'].notnull())]

In [28]:
# Створити InvoiceMonth
# you code 
pd.to_datetime(df["InvoiceDate"])
df["InvoiceMonth"] = df['InvoiceDate'].dt.to_period("M").dt.to_timestamp()
df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01


In [29]:
# Створити CohortMonth
df["CohortMonth"] = (
    df.groupby("CustomerID")["InvoiceDate"].transform("min").dt.to_period("M").dt.to_timestamp()
)



In [30]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01


In [31]:
# Формування час використання 
year_diff = df["InvoiceMonth"].dt.year - df["CohortMonth"].dt.year
month_diff = df["InvoiceMonth"].dt.month - df["CohortMonth"].dt.month

df["CohortIndex"] = year_diff * 12 + month_diff + 1


In [32]:
df.sample(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth,CohortIndex
349829,567533,21430,SET/3 RED GINGHAM ROSE STORAGE BOX,4,2011-09-21 09:45:00,3.75,17651.0,United Kingdom,2011-09-01,2011-06-01,4
101485,544920,22990,COTTON APRON PANTRY DESIGN,6,2011-02-24 16:31:00,4.95,13081.0,United Kingdom,2011-02-01,2010-12-01,3


In [45]:
# Обираємо топ країни
top_countries = (
    df.groupby('Country')['CustomerID'].nunique().sort_values(ascending=False).nlargest(5)
)
print(top_countries)
df_geo = df[df["Country"].isin(top_countries.index)]


Country
United Kingdom    3920
Germany             94
France              87
Spain               30
Belgium             25
Name: CustomerID, dtype: int64


In [44]:
df_geo['Country'].value_counts()

Country
United Kingdom    349203
Germany             9025
France              8326
Spain               2479
Belgium             2031
Name: count, dtype: int64

In [49]:
cohort_counts_geo = pd.pivot_table( # щоб створити зведену таблицю
        df_geo,
        index = 'InvoiceMonth', # обрати індекси
        columns = 'CohortIndex', # обрати колонку
        values = 'CustomerID', # обрати значення 
        aggfunc="nunique" # aggregation
    
)

In [50]:
# Рахуємо retention
cohort_sizes_geo = cohort_counts_geo.iloc[:, 0]
cohort_sizes_geo
retention_geo = cohort_counts_geo.divide(cohort_sizes_geo, axis=0)


In [51]:
# Будуємо heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(
    retention_geo.loc["United Kingdom"],
    annot=True,
    fmt=".0%",

)
plt.title("Retention Cohorts — United Kingdom")
plt.xlabel("Місяць життя користувача")
plt.ylabel("Когорта")
plt.show()


KeyError: 'United Kingdom'

<Figure size 1200x600 with 0 Axes>

In [36]:
# Revenue/LTV
# revenue_geo = pd.pivot_table(
#     df_geo,
#     index=["Country", "CohortMonth"],
#     columns="CohortIndex",
#     values="Revenue",
#     aggfunc="sum"
# )
# revenue_geo

In [37]:
# ARPU
# arpu_geo = revenue_geo.divide(cohort_counts_geo, axis=0)
# arpu_geo

In [38]:
# LTV
# ltv_geo = arpu_geo.cumsum(axis=1)
# ltv_geo